<a href="https://colab.research.google.com/github/jorguzb/Spark/blob/master/Linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [14]:
import pandas as pd
db=pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv')

In [19]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(db) 
sparkDF.printSchema()
sparkDF.show()


root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: long (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: long (nullable = true)
 |-- tax: long (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)

+--------------------+----+-----+----+-------------------+------------------+-----+------+---+---+-------+------+-----+----+
|                crim|  zn|indus|chas|                nox|                rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+--------------------+----+-----+----+-------------------+------------------+-----+------+---+---+-------+------+-----+----+
|             0.00632|18.0| 2.31|   0| 0.5379999999999999|             6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|  

In [21]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
#dataset = spark.read.csv(sparkDF,inferSchema=True, header =True)

In [22]:
#Input all the features in one vector column
assembler = VectorAssembler(inputCols=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'], outputCol = 'Attributes')
output = assembler.transform(sparkDF)
#Input vs Output
finalized_data = output.select("Attributes","medv")
finalized_data.show()

+--------------------+----+
|          Attributes|medv|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
|[0.03236999999999...|33.4|
|[0.06905,0.0,2.18...|36.2|
|[0.02985,0.0,2.18...|28.7|
|[0.08829,12.5,7.8...|22.9|
|[0.14455,12.5,7.8...|27.1|
|[0.21124,12.5,7.8...|16.5|
|[0.17004,12.5,7.8...|18.9|
|[0.22489,12.5,7.8...|15.0|
|[0.11747,12.5,7.8...|18.9|
|[0.09378,12.5,7.8...|21.7|
|[0.62976,0.0,8.14...|20.4|
|[0.63796000000000...|18.2|
|[0.62739,0.0,8.14...|19.9|
|[1.05393,0.0,8.14...|23.1|
|[0.7842,0.0,8.14,...|17.5|
|[0.80271,0.0,8.14...|20.2|
|[0.7258,0.0,8.14,...|18.2|
+--------------------+----+
only showing top 20 rows



In [23]:
#Split training and testing data
train_data,test_data = finalized_data.randomSplit([0.8,0.2])
regressor = LinearRegression(featuresCol = 'Attributes', labelCol = 'medv')
#Learn to fit the model from training set
regressor = regressor.fit(train_data)
#To predict the prices on testing set
pred = regressor.evaluate(test_data)
#Predict the model
pred.predictions.show()

/content/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+----+------------------+
|          Attributes|medv|        prediction|
+--------------------+----+------------------+
|[0.02055,85.0,0.7...|24.7|25.209452817129254|
|[0.0315,95.0,1.47...|34.9|29.898219591398952|
|[0.0351,95.0,2.68...|48.5|41.988993574630484|
|[0.03584,80.0,3.3...|23.5| 30.49307795090272|
|[0.04203,28.0,15....|22.9| 28.83913091495534|
|[0.04666,80.0,1.5...|30.3| 32.81486046399627|
|[0.04981,21.0,5.6...|23.4|24.051733591356623|
|[0.05646,0.0,12.8...|21.2|20.994614124664388|
|[0.0566,0.0,3.41,...|23.6|30.772692987201605|
|[0.05735,0.0,4.49...|26.6|27.758327422520942|
|[0.06911,45.0,3.4...|30.5|30.056269202332146|
|[0.07022,0.0,4.05...|23.2|25.776108964842958|
|[0.08187,0.0,2.89...|43.8|34.317198856672405|
|[0.08829,12.5,7.8...|22.9|23.332352692670074|
|[0.09178,0.0,4.05...|23.6|29.466716484883747|
|[0.09252,30.0,4.9...|23.3|28.448086896253226|
|[0.09512000000000...|21.4| 23.54612973748441|
|[0.11329000000000...|22.0| 27.36299137958996|
|[0.11504,0.0

In [24]:
#coefficient of the regression model
coeff = regressor.coefficients
#X and Y intercept
intr = regressor.intercept
print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)

The coefficient of the model is : DenseVector([-0.11, 0.0531, -0.0097, 2.4274, -18.3299, 3.3767, 0.0118, -1.4907, 0.327, -0.014, -0.9667, 0.0082, -0.5117])
The Intercept of the model is : 40.065599


In [25]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="rmse")
# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)
# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)
# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)
# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

/content/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


RMSE: 5.575
MSE: 31.075
MAE: 3.799
r2: 0.686


In [ ]:
test_pred = regressor.evaluate(test_data)
print (pred)

In [26]:
print(test_data)

DataFrame[Attributes: vector, medv: double]
